In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [ ]:
style = {'description_width': '180px'}
layout = widgets.Layout(width='400px')

apiUrl = widgets.Text(
    value='http://localhost:8000',
    description='API Endpoint:',
    style=style,
    layout=layout
)

gdp = widgets.FloatText(
    value=25000,
    description='GDP (Billions):',
    style=style,
    layout=layout
)

cpi = widgets.FloatText(
    value=300,
    description='CPI:',
    style=style,
    layout=layout
)

unrate = widgets.FloatSlider(
    value=5.5,
    min=0,
    max=20,
    step=0.1,
    description='Unemployment Rate (%):',
    style=style,
    layout=layout
)

fedfunds = widgets.FloatSlider(
    value=4.5,
    min=0,
    max=10,
    step=0.1,
    description='Federal Funds Rate (%):',
    style=style,
    layout=layout
)

mortgage = widgets.FloatSlider(
    value=6.5,
    min=2,
    max=10,
    step=0.1,
    description='Mortgage Rate (%):',
    style=style,
    layout=layout
)

houst = widgets.FloatText(
    value=1400,
    description='Housing Starts (K):',
    style=style,
    layout=layout
)

horizon = widgets.IntSlider(
    value=12,
    min=1,
    max=36,
    description='Forecast Horizon (mo):',
    style=style,
    layout=layout
)

crisisDetection = widgets.Checkbox(
    value=True,
    description='Enable Crisis Detection',
    style={'description_width': 'auto'}
)

predictBtn = widgets.Button(
    description='Generate Prediction',
    button_style='primary',
    icon='rocket',
    layout=widgets.Layout(width='400px', height='50px')
)

statusLabel = widgets.HTML(value='<b style="color: #999;">Ready</b>')

output = widgets.Output()

In [ ]:
def onPredictClick(b):
    with output:
        clear_output(wait=True)
        
        statusLabel.value = '<b style="color: #f39c12;">Running prediction...</b>'
        
        payload = {
            'data': [{
                'GDP': gdp.value,
                'CPIAUCSL': cpi.value,
                'UNRATE': unrate.value,
                'FEDFUNDS': fedfunds.value,
                'MORTGAGE30US': mortgage.value,
                'HOUST': houst.value
            }],
            'horizon': horizon.value,
            'crisisDetection': crisisDetection.value
        }
        
        try:
            response = requests.post(f'{apiUrl.value}/predict', json=payload, timeout=60)
            
            if response.status_code == 200:
                result = response.json()
                
                statusLabel.value = '<b style="color: #27ae60;">✓ Prediction Complete</b>'
                
                display(widgets.HTML(f'<h2 style="color: #3498db;">Prediction Results</h2>'))
                
                metricsHtml = f'<div style="display: flex; gap: 20px; margin: 20px 0;">'
                metricsHtml += f'<div style="background: #2c3e50; padding: 20px; border-radius: 8px; text-align: center; flex: 1;"><div style="color: #95a5a6; font-size: 14px;">Forecast Horizon</div><div style="color: #3498db; font-size: 32px; font-weight: bold;">{result["horizon"]}m</div></div>'
                
                if crisisDetection.value:
                    crisisLevel = result.get('crisisLevel', 'N/A')
                    crisisScore = result.get('crisisScore', 0)
                    
                    colorMap = {'HIGH': '#e74c3c', 'MEDIUM': '#f39c12', 'LOW': '#27ae60', 'DEMO': '#95a5a6'}
                    color = colorMap.get(crisisLevel, '#95a5a6')
                    
                    metricsHtml += f'<div style="background: #2c3e50; padding: 20px; border-radius: 8px; text-align: center; flex: 1;"><div style="color: #95a5a6; font-size: 14px;">Crisis Level</div><div style="color: {color}; font-size: 32px; font-weight: bold;">{crisisLevel}</div></div>'
                    metricsHtml += f'<div style="background: #2c3e50; padding: 20px; border-radius: 8px; text-align: center; flex: 1;"><div style="color: #95a5a6; font-size: 14px;">Crisis Score</div><div style="color: #3498db; font-size: 32px; font-weight: bold;">{crisisScore:.3f}</div></div>'
                
                metricsHtml += '</div>'
                display(widgets.HTML(metricsHtml))
                
                predictions = result['predictions']
                months = list(range(1, len(predictions) + 1))
                
                fig = go.Figure()
                fig.add_trace(go.Scatter(
                    x=months,
                    y=predictions,
                    mode='lines+markers',
                    name='Predicted Values',
                    line=dict(color='#3498db', width=3),
                    marker=dict(size=10, color='#3498db'),
                    fill='tozeroy',
                    fillcolor='rgba(52, 152, 219, 0.1)'
                ))
                
                fig.update_layout(
                    title=dict(text='Housing Price Forecast', font=dict(size=24, color='#ecf0f1')),
                    xaxis_title='Months Ahead',
                    yaxis_title='Predicted Value',
                    template='plotly_dark',
                    height=500,
                    hovermode='x unified',
                    paper_bgcolor='#1a1a1a',
                    plot_bgcolor='#2c3e50'
                )
                
                display(fig)
                
                if result.get('recommendations'):
                    display(widgets.HTML('<h3 style="color: #3498db; margin-top: 30px;">Policy Recommendations</h3>'))
                    recsHtml = '<ul style="list-style: none; padding: 0;">'
                    for i, rec in enumerate(result['recommendations'], 1):
                        recsHtml += f'<li style="background: #2c3e50; padding: 15px; margin: 10px 0; border-left: 4px solid #3498db; border-radius: 4px;">{i}. {rec}</li>'
                    recsHtml += '</ul>'
                    display(widgets.HTML(recsHtml))
            else:
                statusLabel.value = f'<b style="color: #e74c3c;">✗ API Error: {response.status_code}</b>'
                display(widgets.HTML(f'<div style="background: #e74c3c; color: white; padding: 15px; border-radius: 8px;">Error {response.status_code}: {response.text}</div>'))
                
        except Exception as e:
            statusLabel.value = '<b style="color: #e74c3c;">✗ Connection Error</b>'
            display(widgets.HTML(f'<div style="background: #e74c3c; color: white; padding: 15px; border-radius: 8px;">Error: {str(e)}</div>'))

predictBtn.on_click(onPredictClick)

In [ ]:
header = widgets.HTML(
    value='<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 12px; margin-bottom: 20px;"><h1 style="color: white; margin: 0; font-size: 36px;">🏠 Housing Crisis Prediction</h1><p style="color: rgba(255, 255, 255, 0.9); margin: 10px 0 0 0; font-size: 18px;">Multi-Modal Ensemble Framework for Real-Time Analysis</p></div>'
)

economicBox = widgets.VBox([
    widgets.HTML('<h3 style="color: #3498db;">Economic Indicators</h3>'),
    gdp, cpi, unrate, fedfunds
], layout=widgets.Layout(padding='20px', background='#2c3e50', border_radius='12px'))

housingBox = widgets.VBox([
    widgets.HTML('<h3 style="color: #3498db;">Housing Indicators</h3>'),
    mortgage, houst, horizon, crisisDetection
], layout=widgets.Layout(padding='20px', background='#2c3e50', border_radius='12px'))

configBox = widgets.VBox([
    widgets.HTML('<h3 style="color: #3498db;">Configuration</h3>'),
    apiUrl,
    widgets.HTML('<div style="margin: 20px 0;"></div>'),
    predictBtn,
    widgets.HTML('<div style="margin: 10px 0;"></div>'),
    statusLabel
], layout=widgets.Layout(padding='20px', background='#2c3e50', border_radius='12px'))

inputsBox = widgets.HBox(
    [economicBox, housingBox, configBox],
    layout=widgets.Layout(gap='20px', margin='20px 0')
)

display(header)
display(inputsBox)
display(output)